## Getting Started: Library Installations and Setup
Alright, let's get those dependencies sorted out. We'll need a bunch of libraries to kickstart our RAG journey. Don't worry; we've got you covered with a simple pip install command:

In [ ]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                trl==0.4.7
!pip install -q langchain \
                sentence_transformers \
                faiss-cpu \
                pypdf
!pip install -U transformers

## Importing the Tools of the Trade
Now that we've got our libraries in place, let's bring in the heavy artillery. We'll be working with PyTorch, Transformers, and a dash of LangChain magic to weave our RAG masterpiece.

In [ ]:
import os
import torch
from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  BitsAndBytesConfig,
  pipeline
)

from transformers import BitsAndBytesConfig

from langchain.text_splitter import CharacterTextSplitter

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
import transformers
import nest_asyncio

nest_asyncio.apply()
from tqdm.notebook import tqdm
from langchain_community.document_loaders import PyPDFLoader

## Loading the Model and Tokenizer
Ah, the heart and soul of our RAG application—Mistral-7B. Let's fire up the model and tokenizer, but wait, let's do it efficiently to fit it all on a single GPU. We're all about optimization here!

In [ ]:
# Load Tokenizer
model_name='mistralai/Mistral-7B-Instruct-v0.2'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("Tokenizer loaded !!")

In [ ]:
## bitsandbytes parameters
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [ ]:
# Load Model
model = AutoModelForCausalLM.from_pretrained(
        model_name,
            quantization_config=bnb_config,
        )

Lets test our vanilla model on a basic question

In [ ]:
def get_vanilla_response(question, tokenizer, model):
  inputs = tokenizer.encode_plus(f"[INST] {question} [/INST]", return_tensors="pt")['input_ids'].to('cuda')

  generated_ids = model.generate(inputs,
                                max_new_tokens=1000,
                                do_sample=True)
  output = tokenizer.batch_decode(generated_ids)
  return output[0]

print(get_vanilla_response(question = "What changes do I need in my personality to make friends easily?", tokenizer = tokenizer, model = model))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What changes do I need in my personality to make friends easily? [/INST] Making friends can be a challenge for some people, and it often requires some adjustments to our personality traits and social behaviors. Here are some suggestions for changes that may help you make friends more easily:

1. Be open and approachable: People are more likely to be drawn to those who are friendly and welcoming. Smile often, make eye contact, and show genuine interest in others.
2. Be a good listener: Most people love to talk about themselves and share their stories. Be an attentive listener and show empathy when others speak. This will make them feel valued and appreciated.
3. Be patient and persistent: Making friends takes time and effort. Don't get discouraged if it takes a while to build a strong connection. Keep putting yourself out there and be persistent.
4. Be positive and optimistic: Having a positive attitude and a good sense of humor can help attract people to you. People are more

## Crafting the RAG Pipeline
Now comes the fun part. Putting it all together to create our RAG pipeline. We'll be using LangChain to orchestrate the retrieval and generation process seamlessly.

In [ ]:
# Create a text generation pipeline
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=10000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)



### Psychology Book List

1. "Thinking, Fast and Slow" by Daniel Kahneman:
This book delves into the two systems of thinking, providing insights into cognitive biases and decision-making processes.

2. "Influence: The Psychology of Persuasion" by Robert B. Cialdini:
Cialdini explores the principles of influence, shedding light on psychological triggers that impact human behavior.

3. "Predictably Irrational" by Dan Ariely:
Ariely's book explores the irrational behaviors that influence decision-making, providing valuable perspectives on human psychology.

4. "Emotional Intelligence" by Daniel Goleman:
This book emphasizes the importance of emotional intelligence in understanding oneself and others, a crucial aspect of human psychology.

5. "The Social Animal" by Elliot Aronson:
Aronson's book covers a wide range of topics in social psychology, offering insights into human behavior in social contexts.

6. "Nudge" by Richard H. Thaler and Cass R. Sunstein:
Thaler and Sunstein discuss how subtle nudges can influence decision-making, combining insights from psychology and economics.

In [ ]:
# add Book paths from Google Drive
pdf_paths = ['/content/drive/MyDrive/Blogs/psychology-gpt/Dan Ariely - Predictably Irrational_ The Hidden Forces That Shape Our Decisions-HarperCollins (2008).pdf',
             '/content/drive/MyDrive/Blogs/psychology-gpt/Daniel Goleman - Emotional Intelligence_ Why it Can Matter More Than IQ-Bloomsbury (2009).pdf',
             '/content/drive/MyDrive/Blogs/psychology-gpt/Daniel Kahneman - Thinking, Fast and Slow  .pdf',
             '/content/drive/MyDrive/Blogs/psychology-gpt/Elliot Aronson - The Social Animal. Tenth Edition-Worth Publishers (2007).pdf',
             '/content/drive/MyDrive/Blogs/psychology-gpt/Richard H. Thaler, Prof. Cass R. Sunstein - Nudge_ Improving Decisions About Health, Wealth, and Happiness-Yale University Press (2008).pdf',
             '/content/drive/MyDrive/Blogs/psychology-gpt/Robert B. Cialdini - Influence_ The Psychology of Persuasion (Collins Business Essentials) (2007).pdf']
docs = []
for pdf_path in tqdm(pdf_paths):
  loader = PyPDFLoader(pdf_path)
  pages = loader.load_and_split()
  docs = docs+pages[8:-10]

# Load chunked documents into the FAISS index
db = FAISS.from_documents(docs,HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))
retriever = db.as_retriever()


  0%|          | 0/6 [00:00<?, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Create prompt template
prompt_template = """
### [INST] Instruction: Answer the question based on your human psychology knowledge, you can also use this auxillary knowledge to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)



## Let's Test Drive Our RAG
Enough with the setup; let's put our RAG to the test! We'll throw some thought-provoking questions its way and see how it fares.

In [ ]:
op = rag_chain.invoke("What changes do I need in my personality to make friends easily?")
print(op['text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the information provided in the documents, here are some suggestions for developing social skills to make friends more easily:

1. Be attuned to others' needs and feelings: Strive for a balance between being true to yourself and being sensitive to others' needs. This will help you avoid becoming a social chameleon, who may win popularity but sacrifices true satisfaction.
2. Honesty and authenticity: Be genuine in your interactions with others. People appreciate authenticity and are more likely to form lasting relationships with those who are true to themselves.
3. Active listening: Pay attention to what others are saying and respond appropriately. This shows respect and interest in their thoughts and experiences.
4. Empathy: Try to understand others' perspectives and emotions. This will help you connect with them on a deeper level and build stronger relationships.
5. Adaptability: Be flexible in your interactions with others. This means being able to adjust your behavior to f

In [ ]:
# testing on some tricky questions
question = "I constantly have the urge of getting into violence, am I normal and how do I get rid of this?"
op_v =get_vanilla_response(question, tokenizer, model)
print(op_v)
print('===============================================================================')
op = rag_chain.invoke(question)
print(op['text'])
print('================================ Next Question ================================')
question = "How can I find a sense of identity and belonging when I feel lost and disconnected from myself?"
op_v =get_vanilla_response(question, tokenizer, model)
print(op_v)
print('===============================================================================')
op = rag_chain.invoke(question)
print(op['text'])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] I constantly have the urge of getting into violence, am I normal and how do I get rid of this? [/INST] Having occasional urges or thoughts about violence is not uncommon, but if these urges persist and are causing you significant distress or worry, it's important to reach out to a mental health professional. This could be a sign of an underlying mental health condition, such as anger management issues or interpersonal violence.

It's important to remember that feeling angry or having violent thoughts is not the same as acting on them. Violence is never an appropriate response to conflicts or issues, and can have serious consequences for both you and others.

Some things that may help reduce the frequency or intensity of violent urges include:

1. Identifying and addressing the root causes of your anger or violence. This could be related to relationships, work, financial issues, or other stressors.
2. Learning healthy coping mechanisms for managing anger and stress, such as d

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the information provided in the document, it appears that your feelings of urging towards violence could potentially be influenced by various factors including media violence, anxiety, and social learning. However, it's important to note that everyone experiences different emotions and thoughts, and what may seem unusual to one person may be normal for another.

Regarding the first reason mentioned in the document, exposure to media violence, it's common for people to feel more aggressive after watching violent content. This is due to several reasons such as desensitization to violence, identification with the characters, and learning new ways to behave aggressively. However, it's essential to remember that acting on these impulses is not necessary, and it's possible to manage these feelings in healthy ways.

Secondly, anxiety can also contribute to feelings of aggression. Anxiety can manifest in various ways, including physical symptoms such as stomach problems, and emotiona

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] How can I find a sense of identity and belonging when I feel lost and disconnected from myself? [/INST] Finding a sense of identity and belonging when you feel lost and disconnected from yourself can be a challenging process, but there are several steps you can take to help yourself:

1. Self-Reflection: Take some time to reflect on who you are, what you value, and what makes you unique. Consider your strengths, passions, and interests.
2. Connect with Others: Reach out to friends, family, or community groups that share similar interests or experiences. Building strong relationships with others can provide a sense of belonging and support.
3. Practice Self-Care: Prioritize taking care of yourself, both physically and emotionally. Engage in activities that bring you joy and help you feel good about yourself.
4. Set Goals: Establish short-term and long-term goals that align with your values and sense of identity. Achieving these goals can provide a sense of purpose and directi